In [1]:
import torch
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import wandb

In [2]:
!python3 -m wandb login eb7b1964fb84cd81de96b2a273ecf2bb6254aeac

wandb: Appending key for api.wandb.ai to your netrc file: /home/alexeyorlov53/.netrc


In [2]:
filename = 'ecfp0'
samples_count1 = '10M'
model_name1 = f'molberto_{filename}_{samples_count1}'

In [3]:
batch_size = 100

In [4]:
gpu_number=0

### Upload and Split Dataset

In [5]:
dataframe = pd.read_csv("BBBP-2k-ecfp.csv")

In [7]:
# dataframe = dataframe.drop(columns=['Unnamed: 0', 'Smiles', 'ecfp2', 'ecfp3'])

In [6]:
def preprocess_data_dataset(df, column):
    for row in tqdm(range(len(df))):
        str_ints = eval(df.iloc[row][column])
        str_fingerprint = ' '.join(str_ints[0])
        df.at[row, column] = str_fingerprint

In [7]:
preprocess_data_dataset(dataframe, 'ecfp')

  0%|          | 0/1945 [00:00<?, ?it/s]

In [10]:
dataframe

,Smiles,ecfp,target
0,[N+](=NCC(=O)N[C@@H]([C@H](O)C1=CC=C([N+]([O-]...,849271271 847336149 2245384272 2246699815 8649...,1
1,C1=C(OC)C(=CC2=C1C(=[N+](C(=C2CC)C)[NH-])C3=CC...,3218693969 3217380708 864674487 2246728737 321...,1
2,[O+]1=N[N](C=C1[N-]C(NC2=CC=CC=C2)=O)C(CC3=CC=...,3189554341 2041434490 2092489639 3218693969 32...,1
3,[C@H]3([N]2C1=C(C(=NC=N1)N)N=C2)[C@@H]([C@@H](...,2976033787 2092489639 3217380708 3217380708 32...,1
4,C1=C(Cl)C(=C(C2=C1NC(=O)C(N2)=O)[N+](=O)[O-])Cl,3218693969 3217380708 1016841875 3217380708 32...,1
...,...,...,...
1940,CCN1CCN(C(=O)N[C@@H](C(=O)N[C@H]2[C@H]3SCC(=C(...,2246728737 2245384272 2092489639 2968968094 29...,1
1941,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,2246728737 3217380708 3189457552 2041434490 32...,1
1942,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,2968968094 2968968094 2968968094 2092489639 29...,1
1943,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,3217380708 3217380708 3217380708 2092489639 29...,1


In [11]:
# print('Percentage on NaNs:')
# dataframe.isna().mean()

In [12]:
# rows_with_nans = dataframe['Molecular Weight'].isna() | \
#                  dataframe['Bioactivities'].isna() | \
#                  dataframe['AlogP'].isna() | \
#                  dataframe['Polar Surface Area'].isna() | \
#                  dataframe['CX Acidic pKa'].isna() | \
#                  dataframe['CX Basic pKa'].isna()
# print(f'Count of rows without NaNs: {dataframe.shape[0] - dataframe.loc[rows_with_nans].shape[0]}')

In [13]:
# remove 2 last properties to reduce NaN counts
# molecular_properties = ['Molecular Weight', 'Bioactivities', 'AlogP', 'Polar Surface Area']
# dataframe = dataframe.drop(columns=['CX Acidic pKa', 'CX Basic pKa'])

In [14]:
# drop NaN's
# dataframe = dataframe.dropna().reset_index(drop=True)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name1)

tokenizer.model_max_len=512

In [10]:
dataframe['ecfp'][0]

'849271271 847336149 2245384272 2246699815 864942730 847961216 2245273601 2245273601 864662311 3217380708 3218693969 3218693969 3217380708 848127915 864942795 864942730 3218693969 3218693969 2245384272 864662311 847433129'

In [ ]:
tokens = tokenizer(dataframe['ecfp'][0], truncation=True, max_length=512, padding='max_length')

In [ ]:
print(tokens)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["ecfp"], truncation=True, max_length=512, padding='max_length')

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

In [16]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(dataframe)
train_testvalid = dataset.train_test_split(test_size=0.2, seed=15)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=15)

# 10% for test, 10 for validation, 80% for train
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

dataset

DatasetDict({
    train: Dataset({
        features: ['Smiles', 'ecfp', 'target'],
        num_rows: 1556
    })
    test: Dataset({
        features: ['Smiles', 'ecfp', 'target'],
        num_rows: 195
    })
    validation: Dataset({
        features: ['Smiles', 'ecfp', 'target'],
        num_rows: 194
    })
})

### Tokenize Data

In [17]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name1)

tokenizer.model_max_len=512

In [18]:
def tokenize(batch):
  return tokenizer(batch["ecfp"], truncation=True, max_length=512, padding='max_length')

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/1556 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Map:   0%|          | 0/194 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Smiles', 'ecfp', 'target', 'input_ids', 'attention_mask'],
        num_rows: 1556
    })
    test: Dataset({
        features: ['Smiles', 'ecfp', 'target', 'input_ids', 'attention_mask'],
        num_rows: 195
    })
    validation: Dataset({
        features: ['Smiles', 'ecfp', 'target', 'input_ids', 'attention_mask'],
        num_rows: 194
    })
})

In [19]:
columns = ["input_ids", "attention_mask"]
columns.extend(['target']) # our labels
print(columns)
tokenized_dataset.set_format('torch', columns=columns)

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

['input_ids', 'attention_mask', 'target']


### Create Transformer Model

In [20]:
from transformers import AutoModel, AutoConfig

class MolecularPropertiesClassification(torch.nn.Module):
    def __init__(self, model_name1):
        super(MolecularPropertiesClassification, self).__init__()

        config1 = AutoConfig.from_pretrained(model_name1)
        self.transformer1 = AutoModel.from_pretrained(model_name1, config=config1)
        # removing last layer of transformer
        self.transformer1.pooler = torch.nn.Identity()
        # freezing transformer weights
        for param in self.transformer1.parameters():
            param.requires_grad = False
            
        self.linear1 = torch.nn.Linear(768, 768, bias=True)
        self.linear2 = torch.nn.Linear(768, 2, bias=True)

    def forward(self, input_ids = None, attention_mask=None):
        outputs1 = self.transformer1(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state1 = outputs1[0]
        
        first_linear_out = self.linear1(last_hidden_state1[:, 0, : ].view(-1, 768))
        logits = self.linear2(torch.nn.functional.sigmoid(first_linear_out))

        return logits
        

### Create PyTorch DataLoader

In [21]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset['train'], shuffle = True, batch_size = batch_size, collate_fn = data_collator
)

eval_dataloader = DataLoader(
    tokenized_dataset['validation'], shuffle = True, batch_size = batch_size, collate_fn = data_collator
)

test_dataloader = DataLoader(
    tokenized_dataset['test'], shuffle = True, batch_size = batch_size, collate_fn = data_collator
)

In [22]:
device = torch.device("cuda", index=gpu_number) if torch.cuda.is_available() else torch.device('cpu')

model = MolecularPropertiesClassification(model_name1).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at molberto_ecfp0_10M and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
model

MolecularPropertiesClassification(
  (transformer1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [24]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epoch = 100

num_training_steps = num_epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

loss_func = torch.nn.CrossEntropyLoss()

/home/alexeyorlov53/anaconda3/envs/myenv/lib/python3.9/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
wandb.init(
    project="efcp_transformer",
    name='ECFP-BERT-' + samples_count1 + "-BBBP",
    config={}
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: orlov-aleksei53 (moleculary-ai). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible


### Training

In [26]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epoch * len(eval_dataloader)))

for epoch in range(num_epoch):
    model.train()
    total_pred_labels = []
    total_true_labels = []
    epoch_loss = 0
    for batch in train_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask'] }
        batch['target'] = batch['target'].to(device)
        
        logits = model(**input_batch)
        
        loss = loss_func(logits.view(-1, 2), batch['target'].view(-1))
        loss.backward()
        epoch_loss += loss.item()
        
        pred_labels = torch.argmax(logits, dim=-1)
        true_labels = batch['target']
        total_pred_labels.append(pred_labels)
        total_true_labels.append(true_labels)
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

    total_pred_labels = torch.cat(total_pred_labels).cpu().detach().numpy()
    total_true_labels = torch.cat(total_true_labels).cpu().detach().numpy()
    
    wandb.log({"loss/train": epoch_loss / len(train_dataloader)})
    wandb.log({"accuracy/train": accuracy_score(total_true_labels, total_pred_labels)})
    wandb.log({"f1/train": f1_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"precision/train": precision_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"recall/train": recall_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"roc_auc_score/train": roc_auc_score(total_true_labels, total_pred_labels)})


    model.eval()
    total_pred_labels = []
    total_true_labels = []
    epoch_loss = 0
    for batch in eval_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask'] }
        batch['target'] = batch['target'].to(device)
        
        with torch.no_grad():
            logits = model(**input_batch)
            loss = loss_func(logits.view(-1, 2), batch['target'].view(-1))
            epoch_loss += loss.item()

            pred_labels = torch.argmax(logits, dim=-1)
            true_labels = batch['target']
            total_pred_labels.append(pred_labels)
            total_true_labels.append(true_labels)
        
        progress_bar_eval.update(1)

    total_pred_labels = torch.cat(total_pred_labels).cpu().detach().numpy()
    total_true_labels = torch.cat(total_true_labels).cpu().detach().numpy()
    
    wandb.log({"loss/validation": epoch_loss / len(eval_dataloader)})
    wandb.log({"accuracy/validation": accuracy_score(total_true_labels, total_pred_labels)})
    wandb.log({"f1/validation": f1_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"precision/validation": precision_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"recall/validation": recall_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"roc_auc_score/validation": roc_auc_score(total_true_labels, total_pred_labels)})

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [27]:
def test_loop():
    model.eval()
    total_pred_labels = []
    total_true_labels = []
    epoch_loss = 0
    for batch in test_dataloader:
        input_batch = { k: v.to(device) for k, v in batch.items() if k in ['input_ids', 'attention_mask'] }
        batch['target'] = batch['target'].to(device)
        
        with torch.no_grad():
            logits = model(**input_batch)
            loss = loss_func(logits.view(-1, 2), batch['target'].view(-1))
            epoch_loss += loss.item()

            pred_labels = torch.argmax(logits, dim=-1)
            true_labels = batch['target']
            total_pred_labels.append(pred_labels)
            total_true_labels.append(true_labels)
        
        progress_bar_eval.update(1)

    total_pred_labels = torch.cat(total_pred_labels).cpu().detach().numpy()
    total_true_labels = torch.cat(total_true_labels).cpu().detach().numpy()
    
    wandb.log({"loss/test": epoch_loss / len(test_dataloader)})
    wandb.log({"accuracy/test": accuracy_score(total_true_labels, total_pred_labels)})
    wandb.log({"f1/test": f1_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"precision/test": precision_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"recall/test": recall_score(total_true_labels, total_pred_labels, average='micro')})
    wandb.log({"roc_auc_score/test": roc_auc_score(total_true_labels, total_pred_labels)})


In [28]:
test_loop()

In [29]:
wandb.finish()

accuracy/test,▁
accuracy/train,▁▁▂▃▅▆▆▆▆▇▇▇▇██▇███▇█▇▇█▇▇█▇▇█████▇█████
accuracy/validation,▁▁▂▄▇▇██████▇█▇█▇█▇█████▇████████▇█▇████
f1/test,▁
f1/train,▁▁▂▃▅▆▆▆▆▇▇▇▇██▇███▇█▇▇█▇▇█▇▇█████▇█████
f1/validation,▁▁▂▄▇▇██████▇█▇█▇█▇█████▇████████▇█▇████
loss/test,▁
loss/train,█▇▅▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss/validation,█▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
precision/test,▁
precision/train,▁▁▂▃▅▆▆▆▆▇▇▇▇██▇███▇█▇▇█▇▇█▇▇█████▇█████


In [30]:
torch.cuda.empty_cache()